In [3]:
"""
CELL 1: SETUP
Description: Load configuration and all VAANI functions
"""

# Load config
%run vyapar_config.ipynb

# Load functions
%run vaani_functions.ipynb

print("="*60)
print("🎮 VAANI PLAYGROUND LOADED")
print("="*60)
print("✅ Config loaded")
print("✅ Functions loaded")
print("✅ Ready to test!")

✅ Libraries imported successfully
✅ Master Registry Link configured
📍 Link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQdOVYDNLuMG...

🔄 Testing registry load...
✅ Registry loaded successfully: 8 items found

📋 Available items in registry:
  1. default_model
  2. openai_api_key
  3. anthropic_api_key
  4. master_registry_link
  5. usage_data
  6. dropoff_analysis
  7. user_research
  8. excel_db_1000_items

🧪 Testing get_item() function:

1. Testing with 'default_model':
✅ Registry loaded successfully: 8 items found
✅ Retrieved 'default_model'
   Result: {'model': 'claude-sonnet-4-20250514', 'temperature': 0.3, 'max_tokens': 1000, 'provider': 'anthropic'}

2. Testing with non-existent item:
✅ Registry loaded successfully: 8 items found
❌ Item 'this_does_not_exist' not found in registry
💡 Available items: default_model, openai_api_key, anthropic_api_key, master_registry_link, usage_data, dropoff_analysis, user_research, excel_db_1000_items

3. Testing with Sarvam API (might not ha

In [8]:
"""
CELL 2: QUICK SINGLE TEST
Description: Test a single input with the complete flow
"""

# ⬇️ MODIFY THIS INPUT AND RUN ⬇️
test_input = "chai samosa 140 rupees"

# ⬇️ OPTIONALLY SPECIFY TRANSACTION TYPE (None = auto-detect) ⬇️
transaction_type = None  # Options: None, 'expense', 'sale', 'purchase', 'payment_in', 'payment_out'

print("="*60)
print(f"📝 Testing: '{test_input}'")
print("="*60)

result = route_with_intent(test_input, transaction_type=transaction_type)

print(f"\n📊 RESULT:")
print(f"Status: {result.get('status')}")

if result.get('status') == 'success':
    print(f"Transaction Type: {result.get('transaction_type')}")
    print(f"\n✅ Extracted Data:")
    extraction = result.get('extraction', {})
    for key, value in extraction.items():
        if key not in ['raw_response', 'transaction_type']:
            print(f"   {key}: {value}")
    
    # Show timing in milliseconds
    if 'time_taken' in extraction:
        ms = extraction['time_taken'] * 1000
        print(f"\n⏱️  Processing Time: {ms:.0f} ms ({extraction['time_taken']:.2f}s)")

elif result.get('status') == 'not_relevant':
    print(f"⚠️  {result.get('message')}")

else:
    print(f"❌ Error: {result.get('error')}")

print("\n" + "="*60)

📝 Testing: 'chai samosa 140 rupees'
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

📊 RESULT:
Status: None
❌ Error: Extraction failed: Expecting value: line 1 column 1 (char 0)



In [5]:
"""
CELL 3: BATCH TEST MULTIPLE INPUTS
Description: Test multiple inputs at once with summary table
"""

import pandas as pd

# ⬇️ ADD YOUR TEST INPUTS HERE ⬇️
test_inputs = [
    # Expenses
    "chai samosa 140 rupees",
    "petrol 500 rupees",
    "taxi ke liye 200 diye",
    "delivery charges 50",
    
    # Sales
    "Sharma ji bought 5kg rice for 250 rupees",
    "sold vegetables to Ramesh 300 rupees",
    
    # Not relevant
    "what's the weather today?",
    "hello how are you",
    
    # Complex
    "chai 60, samosa 80, biscuit 20",
    "Mishra aunty ne 500 ka udhar chukaya",
]

print("🧪 BATCH TESTING")
print(f"Testing {len(test_inputs)} inputs...")
print("="*60)

results = []

for idx, inp in enumerate(test_inputs, 1):
    print(f"\n[{idx}/{len(test_inputs)}] Processing...")
    
    result = route_with_intent(inp)
    
    # Build result row
    row = {
        'input': inp[:50] + '...' if len(inp) > 50 else inp,
        'status': result.get('status'),
        'transaction_type': result.get('transaction_type', 'N/A'),
    }
    
    # Add timing
    if 'extraction' in result and 'time_taken' in result['extraction']:
        ms = result['extraction']['time_taken'] * 1000
        row['time_ms'] = f"{ms:.0f}"
    else:
        row['time_ms'] = 'N/A'
    
    # Add extracted data based on transaction type
    if result.get('status') == 'success':
        extraction = result.get('extraction', {})
        
        if result.get('transaction_type') == 'expense':
            row['amount'] = extraction.get('amount', 'N/A')
            row['item'] = extraction.get('item', 'N/A')
            row['category'] = extraction.get('category', 'N/A')
        
        elif result.get('transaction_type') == 'sale':
            row['customer'] = extraction.get('customer_name', 'N/A')
            row['amount'] = extraction.get('amount', 'N/A')
        
        else:
            row['details'] = str(extraction)[:30]
    
    results.append(row)

# Create DataFrame
df_results = pd.DataFrame(results)

print("\n" + "="*60)
print("📊 BATCH TEST RESULTS")
print("="*60)
print(df_results.to_string(index=False))

# Summary statistics
print("\n" + "="*60)
print("📈 SUMMARY")
print("="*60)

total = len(df_results)
success = (df_results['status'] == 'success').sum()
not_relevant = (df_results['status'] == 'not_relevant').sum()
errors = (df_results['status'] == 'error').sum()

print(f"Total Tests: {total}")
print(f"✅ Success: {success} ({success/total*100:.1f}%)")
print(f"⚠️  Not Relevant: {not_relevant} ({not_relevant/total*100:.1f}%)")
print(f"❌ Errors: {errors} ({errors/total*100:.1f}%)")

# Timing stats
if 'time_ms' in df_results.columns:
    valid_times = pd.to_numeric(df_results['time_ms'], errors='coerce').dropna()
    if len(valid_times) > 0:
        print(f"\n⏱️  Timing Stats:")
        print(f"   Average: {valid_times.mean():.0f} ms")
        print(f"   Min: {valid_times.min():.0f} ms")
        print(f"   Max: {valid_times.max():.0f} ms")

# Transaction type breakdown
if 'transaction_type' in df_results.columns:
    print(f"\n📋 By Transaction Type:")
    type_counts = df_results['transaction_type'].value_counts()
    for ttype, count in type_counts.items():
        print(f"   {ttype}: {count}")

print("\n✅ Results saved in 'df_results' DataFrame")

🧪 BATCH TESTING
Testing 10 inputs...

[1/10] Processing...
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

[2/10] Processing...
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

[3/10] Processing...
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

[4/10] Processing...
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 it

In [6]:
"""
CELL 4: MODEL COMPARISON
Description: Compare how different models handle the same input
"""

# ⬇️ INPUT TO TEST ⬇️
test_input = "chai samosa 140 rupees"

# ⬇️ TRANSACTION TYPE (None = auto-detect) ⬇️
transaction_type = "expense"  # Skip intent detection for fair comparison

# ⬇️ MODELS TO COMPARE ⬇️
models_to_test = [
    'claude-sonnet-4-20250514',
    # 'gpt-4o-mini',  # Uncomment when OpenAI key is configured
    # 'gpt-4o',
]

print("🔬 MODEL COMPARISON")
print(f"Input: '{test_input}'")
print(f"Transaction Type: {transaction_type}")
print("="*60)

comparison_results = []

for model in models_to_test:
    print(f"\n🤖 Testing with: {model}")
    print("-"*60)
    
    try:
        result = route_with_intent(test_input, transaction_type=transaction_type, model=model)
        
        if result.get('status') == 'success':
            extraction = result['extraction']
            ms = extraction.get('time_taken', 0) * 1000
            
            print(f"✅ Success")
            print(f"   Amount: {extraction.get('amount')}")
            print(f"   Item: {extraction.get('item')}")
            print(f"   Category: {extraction.get('category')}")
            print(f"   Time: {ms:.0f} ms")
            
            comparison_results.append({
                'model': model,
                'amount': extraction.get('amount'),
                'item': extraction.get('item'),
                'category': extraction.get('category'),
                'time_ms': f"{ms:.0f}"
            })
        else:
            print(f"❌ Failed: {result.get('error', result.get('message'))}")
    
    except Exception as e:
        print(f"❌ Exception: {e}")

# Show comparison table
if comparison_results:
    print("\n" + "="*60)
    print("📊 COMPARISON TABLE")
    print("="*60)
    df_comparison = pd.DataFrame(comparison_results)
    print(df_comparison.to_string(index=False))

🔬 MODEL COMPARISON
Input: 'chai samosa 140 rupees'
Transaction Type: expense

🤖 Testing with: claude-sonnet-4-20250514
------------------------------------------------------------
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
⚠️ Model config 'claude-sonnet-4-20250514' not found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
❌ Failed: Extraction failed: Expecting value: line 1 column 1 (char 0)


In [7]:
"""
CELL 5: TEST BY TRANSACTION TYPE
Description: Test one example of each transaction type
"""

test_cases_by_type = {
    'expense': [
        "chai 60 rupees",
        "petrol 500 rupees",
        "taxi mein 200 lag gaye",
    ],
    'sale': [
        "Sharma ji bought 5kg rice for 250",
        "sold vegetables to Ramesh 300 rupees",
        "customer paid 1000 for order",
    ],
    'purchase': [
        "bought 10kg onions for 500 from supplier",
        "ordered stock of 50 units at 2000",
    ],
    'payment_in': [
        "Ramesh paid his dues 5000 rupees",
        "received payment from ABC Store 10000",
    ],
    'payment_out': [
        "paid supplier 15000 rupees",
        "cleared vendor dues 8000",
    ],
}

print("🧪 TESTING BY TRANSACTION TYPE")
print("="*60)

all_results = []

for trans_type, cases in test_cases_by_type.items():
    print(f"\n📋 {trans_type.upper()}:")
    print("-"*60)
    
    for case in cases:
        result = route_with_intent(case, transaction_type=trans_type)
        
        if result.get('status') == 'success':
            extraction = result['extraction']
            ms = extraction.get('time_taken', 0) * 1000
            print(f"✅ '{case}'")
            print(f"   Time: {ms:.0f} ms")
            
            # Show key fields based on type
            if trans_type == 'expense':
                print(f"   → Amount: {extraction.get('amount')}, Item: {extraction.get('item')}")
            elif trans_type == 'sale':
                print(f"   → Customer: {extraction.get('customer_name')}, Amount: {extraction.get('amount')}")
            elif trans_type == 'purchase':
                print(f"   → Supplier: {extraction.get('supplier_name')}, Amount: {extraction.get('total_amount')}")
            
            all_results.append({
                'type': trans_type,
                'input': case[:40],
                'time_ms': ms,
                'success': True
            })
        else:
            print(f"❌ '{case}'")
            print(f"   Error: {result.get('error', result.get('message'))}")
            all_results.append({
                'type': trans_type,
                'input': case[:40],
                'time_ms': 0,
                'success': False
            })

# Summary by type
print("\n" + "="*60)
print("📊 SUMMARY BY TYPE")
print("="*60)

df_all = pd.DataFrame(all_results)
summary = df_all.groupby('type').agg({
    'success': ['sum', 'count'],
    'time_ms': 'mean'
}).round(0)

print(summary)

🧪 TESTING BY TRANSACTION TYPE

📋 EXPENSE:
------------------------------------------------------------
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
❌ 'chai 60 rupees'
   Error: Extraction failed: Expecting value: line 1 column 1 (char 0)
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
❌ 'petrol 500 rupees'
   Error: Extraction failed: Expecting value: line 1 column 1 (char 0)
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
❌ 'taxi mein 200 lag gaye'
   Error: Extraction failed: Expecting value: line 1 column 1 (char 0)

📋 SALE:
-------------

KeyboardInterrupt: 

In [ ]:
"""
CELL 6: EDGE CASES & ERROR TESTING
Description: Test edge cases to find breaking points
"""

edge_cases = {
    'Missing Information': [
        "spent 500",  # No item
        "bought vegetables",  # No amount
        "paid someone",  # Vague
    ],
    'Ambiguous': [
        "chai and samosa for 140",  # Multiple items, single price
        "around 100 rupees",  # Approximate
        "delivery 50",  # Unclear item
    ],
    'Complex/Long': [
        "Sharma ji ne aaj subah 2 kilo aloo 100 rupees mein liye aur 1 kilo pyaaz 80 mein",
        "taxi from home to office via market took 250 rupees",
    ],
    'Invalid': [
        "minus 50 rupees",  # Negative
        "free chai",  # No money
        "",  # Empty
    ],
    'Not Business': [
        "what's the weather?",
        "hello how are you",
        "tell me a joke",
    ],
}

print("🧪 EDGE CASE TESTING")
print("="*60)

for category, cases in edge_cases.items():
    print(f"\n📋 {category}")
    print("-"*60)
    
    for case in cases:
        if not case:
            continue
            
        result = route_with_intent(case)
        
        status_icon = {
            'success': '✅',
            'not_relevant': '⚠️',
            'error': '❌',
        }.get(result.get('status'), '❓')
        
        print(f"{status_icon} '{case}'")
        print(f"   Status: {result.get('status')}")
        
        if result.get('status') == 'success':
            extraction = result.get('extraction', {})
            ms = extraction.get('time_taken', 0) * 1000
            print(f"   Time: {ms:.0f} ms")
            print(f"   Type: {result.get('transaction_type')}")
        elif result.get('status') == 'not_relevant':
            print(f"   Reason: {result.get('message')}")

In [ ]:
"""
CELL 7: CUSTOM TEST AREA
Description: Quick experimentation - modify and re-run
"""

# ⬇️ YOUR CUSTOM TESTS HERE ⬇️
my_tests = [
    "chai 60 rupees",
    "Sharma ji paid 5000",
]

print("🎯 CUSTOM TESTS")
print("="*60)

for test in my_tests:
    print(f"\nTesting: '{test}'")
    print("-"*40)
    
    result = route_with_intent(test)
    
    print(f"Status: {result.get('status')}")
    print(f"Type: {result.get('transaction_type')}")
    
    if result.get('status') == 'success':
        extraction = result.get('extraction', {})
        ms = extraction.get('time_taken', 0) * 1000
        print(f"Time: {ms:.0f} ms")
        print(f"Data: {extraction}")

In [ ]:
"""
CELL 8: PLAYGROUND SUMMARY
Description: Quick reference guide
"""

print("""
🎮 VAANI PLAYGROUND - QUICK GUIDE
=====================================================

📝 QUICK TESTS:
- Cell 2: Single input test (fastest)
- Cell 7: Custom test area (flexible)

🧪 BATCH TESTS:
- Cell 3: Test multiple inputs at once with table
- Cell 5: Test all transaction types
- Cell 6: Edge cases and error testing

🔬 COMPARISONS:
- Cell 4: Compare different models

💡 TIPS:
- Times are shown in milliseconds (ms)
- Modify test inputs directly in cells
- Use transaction_type parameter to skip intent detection
- Check df_results DataFrame after batch tests

⏱️  TIMING:
- Times include full LLM API call + processing
- Lower is better (aim for <2000ms)

=====================================================
✅ Start with Cell 2 for quick tests! 🚀
""")